In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
import numpy as np
path = './dftpy/src' # If DFTpy is not in PYTHONPATH, change it to the path of where dftpy/src is located.
if path not in sys.path:
    sys.path.insert(0, path)
from dftpy.ions import Ions
from dftpy.field import DirectField
from dftpy.grid import DirectGrid
from dftpy.functional import LocalPseudo, Functional, TotalFunctional
from dftpy.formats import io
from dftpy.math_utils import ecut2nr
from dftpy.time_data import TimeData
from dftpy.optimization import Optimization
from dftpy.mpi import sprint
file1='Al_lda.oe01.recpot'
PP_list = {'Al': './dftpy/examples/DATA/'+file1}

ModuleNotFoundError: No module named 'dftpy'

In [5]:
from ase.build import bulk
atoms = bulk('Al', 'fcc', a=4.05, cubic=True)
ions = Ions.from_ase(atoms)

In [50]:
ions.cell

Cell([7.65339080963956, 7.65339080963956, 7.65339080963956])

In [51]:
nr = ecut2nr(ecut=50, lattice=ions.cell)
nr

array([24, 24, 24], dtype=int32)

In [56]:
grid = DirectGrid(lattice=ions.cell, nr=nr)

In [57]:
rho_ini = DirectField(grid=grid)
rho_ini[:] = ions.get_ncharges()/ions.cell.volume

In [60]:
rho_ini[23,0,0]

0.026768225022109817

In [61]:
PSEUDO = LocalPseudo(grid = grid, ions=ions, PP_list=PP_list)
HARTREE = Functional(type='HARTREE')
XC = Functional(type='XC',name='LDA')
KE = Functional(type='KEDF',name='x_TF_y_vW')

setting key: Al -> ./dftpy/examples/DATA/Al_lda.oe01.recpot


In [62]:
evaluator = TotalFunctional(KE=KE, XC=XC, HARTREE=HARTREE, PSEUDO=PSEUDO)
optimization_options = {'econv' : 1e-6*ions.nat}
opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options,
        optimization_method = 'TN')
%timeit -n1 -r2 rho = opt.optimize_rho(guess_rho=rho_ini)

Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       2.692153511701E+00      2.692154E+00    7.877607E-01    1       1       1.605105E-02    
1       2.507342441701E+00      -1.848111E-01   6.524254E-02    3       1       4.556298E-02    
2       2.502503925420E+00      -4.838516E-03   6.526437E-03    6       1       8.997989E-02    
3       2.502022358505E+00      -4.815669E-04   4.720536E-04    8       1       1.457212E-01    
4       2.501993287131E+00      -2.907137E-05   3.869313E-05    7       1       1.955390E-01    
5       2.501990381972E+00      -2.905159E-06   1.920931E-06    9       1       2.571809E-01    
6       2.501990325788E+00      -5.618362E-08   1.685895E-07    3       1       2.834179E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.3011514616033731
Chemical potential (eV)  : 8.194747891565234
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)     

In [63]:
rho = opt.optimize_rho(guess_rho=rho_ini)

Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       2.692153511701E+00      2.692154E+00    7.877607E-01    1       1       6.994963E-03    
1       2.507342441701E+00      -1.848111E-01   6.524254E-02    3       1       3.571582E-02    
2       2.502503925420E+00      -4.838516E-03   6.526437E-03    6       1       7.786703E-02    
3       2.502022358505E+00      -4.815669E-04   4.720536E-04    8       1       1.412361E-01    
4       2.501993287131E+00      -2.907137E-05   3.869313E-05    7       1       1.918249E-01    
5       2.501990381972E+00      -2.905159E-06   1.920931E-06    9       1       2.508650E-01    
6       2.501990325788E+00      -5.618362E-08   1.685895E-07    3       1       2.780118E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.3011514616033731
Chemical potential (eV)  : 8.194747891565234


In [64]:
rho.integral()

11.999999999999996

In [65]:
rho_g=rho.fft()

In [66]:
rho_g

ReciprocalField([[[ 1.20000000e+01+0.00000000e+00j,
                   -1.06130543e-09+1.56101272e-09j,
                   -2.99238852e-01+7.75019375e-10j,
                    6.57546696e-10+1.90162793e-10j,
                   -3.25982565e-02-1.85593962e-09j,
                   -1.44366329e-10-1.02464838e-10j,
                    1.12494307e-02-1.10861746e-09j,
                   -1.25365292e-10-6.84101032e-11j,
                   -2.05718274e-03-6.55450250e-10j,
                    1.85219733e-10+1.47567464e-10j,
                    2.77238784e-04+6.10654135e-10j,
                   -4.29794915e-10-1.25707546e-10j,
                   -4.83717530e-05+3.92271074e-21j,
                   -4.29794915e-10+1.25707547e-10j,
                    2.77238784e-04-6.10654135e-10j,
                    1.85219732e-10-1.47567464e-10j,
                   -2.05718274e-03+6.55450250e-10j,
                   -1.25365292e-10+6.84101031e-11j,
                    1.12494307e-02+1.10861746e-09j,
            

In [67]:
grid_g = rho_g.grid
grid_g

In [34]:
grid_g.gg

array([[[  0.        ,   0.67398768,   2.69595071,   6.06588909,
          10.78380282,  16.84969191,  24.26355635,  33.02539614,
          43.13521128,  54.59300178,  67.39876763,  54.59300178,
          43.13521128,  33.02539614,  24.26355635,  16.84969191,
          10.78380282,   6.06588909,   2.69595071,   0.67398768],
        [  0.67398768,   1.34797535,   3.36993838,   6.73987676,
          11.4577905 ,  17.52367958,  24.93754402,  33.69938382,
          43.80919896,  55.26698946,  68.07275531,  55.26698946,
          43.80919896,  33.69938382,  24.93754402,  17.52367958,
          11.4577905 ,   6.73987676,   3.36993838,   1.34797535],
        [  2.69595071,   3.36993838,   5.39190141,   8.76183979,
          13.47975353,  19.54564261,  26.95950705,  35.72134684,
          45.83116199,  57.28895249,  70.09471834,  57.28895249,
          45.83116199,  35.72134684,  26.95950705,  19.54564261,
          13.47975353,   8.76183979,   5.39190141,   3.36993838],
        [  6.06588909,

In [35]:
grid_g.gg[0,0,0]=1.

In [68]:
v_h_of_g=rho_g*4*np.pi/grid_g.gg
v_h_of_g[0,0,0]=0.
v_h=v_h_of_g.ifft()

In [69]:
H=HARTREE(rho=rho)

In [70]:
v_h.real-H.potential

DirectField([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  1.08420217e-19,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
              [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         

In [ ]:
#rho.read()